In [1]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
from pathlib import Path
import logging

In [2]:
# Set up basic logging for the notebook
logging.basicConfig(level=logging.INFO, format='[%(asctime)s]: %(message)s:')
logger = logging.getLogger("prepare_base_model_logger")

In [3]:
%pwd

'c:\\Users\\DELL\\cnn-models\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\DELL\\cnn-models'

In [6]:
# Cell 2: Define Configuration Entity for Base Model (for prototyping)
# In a real modular setup, this would be in src/cnn_classifier/entity/config_entity.py
from dataclasses import dataclass

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_batch_size: int # Although not directly used for model architecture, useful for context
    params_is_augmentation: bool # Not directly used for model architecture
    params_num_classes: int
    params_learning_rate: float # Not directly used for model architecture
    params_epochs: int # Not directly used for model architecture
    params_patience: int # Not directly used for model architecture


In [7]:
# Cell 3: Mocking Configuration Manager (for notebook testing)
# In a real modular setup, this would use the actual ConfigurationManager from src/cnn_classifier/config/configuration.py
import yaml
from box import ConfigBox

def read_yaml_mock(path_to_yaml: Path) -> ConfigBox:
    with open(path_to_yaml) as yaml_file:
        content = yaml.safe_load(yaml_file)
        logger.info(f"yaml file: {path_to_yaml} loaded successfully")
        return ConfigBox(content)

def create_directories_mock(path_to_directories: list):
    for path in path_to_directories:
        os.makedirs(path, exist_ok=True)
        logger.info(f"created directory at: {path}")

In [10]:
# Mock paths for notebook
CONFIG_FILE_PATH_MOCK = Path("config/config.yaml")
PARAMS_FILE_PATH_MOCK = Path("params.yaml")

class ConfigurationManagerMock:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH_MOCK,
        params_filepath = PARAMS_FILE_PATH_MOCK):

        self.config = read_yaml_mock(config_filepath)
        self.params = read_yaml_mock(params_filepath)

        create_directories_mock([self.config.artifacts_root])
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        params = self.params

        create_directories_mock([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=list(params.IMAGE_SIZE),
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.IS_AUGMENTATION,
            params_num_classes=params.NUM_CLASSES,
            params_learning_rate=params.LEARNING_RATE,
            params_epochs=params.EPOCHS,
            params_patience=params.PATIENCE
        )
        return prepare_base_model_config    

In [11]:
# Cell 4: Prepare Base Model Component Logic (for prototyping)
# In a real modular setup, this would be in src/cnn_classifier/components/prepare_base_model.py

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        logger.info(f"Using device: {self.device}")

    def get_base_model(self):
        # Load pre-trained ResNet18
        self.model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1) # Use weights=... for pre-trained
        logger.info("Loaded pre-trained ResNet18 model.")

        # Freeze all parameters in the feature extracting layers
        for param in self.model.parameters():
            param.requires_grad = False
        logger.info("Frozen all parameters in the base model.")

        # Save the initial base model (frozen)
        torch.save(self.model.state_dict(), self.config.base_model_path)
        logger.info(f"Initial base model saved to: {self.config.base_model_path}")

    def update_base_model(self):
        # Load the frozen base model (if not already loaded)
        if not hasattr(self, 'model'):
            self.get_base_model() # Ensure base model is loaded and frozen

        # Modify the final classification layer
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, self.config.params_num_classes)
        logger.info(f"Modified final FC layer to have {self.config.params_num_classes} output features.")

        # Move model to device
        self.model.to(self.device)
        logger.info(f"Model moved to {self.device}.")

        # Save the updated base model
        torch.save(self.model.state_dict(), self.config.updated_base_model_path)
        logger.info(f"Updated base model saved to: {self.config.updated_base_model_path}")


In [14]:
# Cell 5: Notebook Execution for Prepare Base Model Stage
try:
    # Initialize mock configuration manager
    config_manager_mock = ConfigurationManagerMock()
    prepare_base_model_config = config_manager_mock.get_prepare_base_model_config()

    # Initialize and run the component
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model() # Save initial frozen model
    prepare_base_model.update_base_model() # Update and save final model

    logger.info("🎉 Prepare Base Model stage completed successfully!")

except Exception as e:
    logger.exception(f"❌ Prepare Base Model stage failed: {e}")
    raise e

[2025-07-20 18:16:50,203]: yaml file: config\config.yaml loaded successfully:
[2025-07-20 18:16:50,205]: yaml file: params.yaml loaded successfully:
[2025-07-20 18:16:50,207]: created directory at: artifacts:
[2025-07-20 18:16:50,208]: created directory at: artifacts/prepare_base_model:
[2025-07-20 18:16:50,209]: Using device: cuda:
[2025-07-20 18:16:50,390]: Loaded pre-trained ResNet18 model.:
[2025-07-20 18:16:50,391]: Frozen all parameters in the base model.:
[2025-07-20 18:16:50,441]: Initial base model saved to: artifacts\prepare_base_model\base_model.pth:
[2025-07-20 18:16:50,442]: Modified final FC layer to have 4 output features.:
[2025-07-20 18:16:50,706]: Model moved to cuda.:
[2025-07-20 18:16:50,785]: Updated base model saved to: artifacts\prepare_base_model\updated_base_model.pth:
[2025-07-20 18:16:50,786]: 🎉 Prepare Base Model stage completed successfully!:
